In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pywt
import glob
import pandas as pd
from pandas.compat import StringIO
from scipy import signal
from scipy import stats
from scipy.signal import sosfiltfilt, butter
import time

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def window_stack(a, stepsize=1, width=3):
    n = int(np.floor((a.shape[1] - width) / stepsize))
    return np.dstack( a[:,i*stepsize:width+i*stepsize] for i in range(0,n) )

def minmax_norm(x):
    return 2*(x - np.min(x))/(np.max(x) - np.min(x)) - 1

# Parameters for spectrogram
fs = 512
n_fft = 512
n_perseg = 128
# Data window length
out_shape = 128
n_step = 64

# Training File location
file_list = sorted(glob.glob('/home/sungmin2/py35_codes/00_AnoGAN/EEG_DATA/test4coderuntime/no/*.txt'))

# Data output path
output_path = '/home/sungmin2/py35_codes/00_AnoGAN/EEG_DATA/test4coderuntime/no/processed/'
os.makedirs(output_path, exist_ok=True) 

In [2]:
%%timeit

# get the start time
st = time.process_time()

for file_name in file_list:
#     print('*'*5 + ' processing: ' + file_name)
    fname_split = file_name.split(".")

    fname_split_id = fname_split[0].split("/")
    fname_id = fname_split_id[-1]

    data_read = pd.read_csv(file_name, delimiter = '\t', chunksize=fs * 20, usecols = ['T9','T10', 'TP9', 'TP10'], 
                        na_values=['T9','T10', 'TP9', 'TP10'], dtype=np.float64 ).read()
    
    T9_signal_data = np.array(data_read['T9'])
    T9_signal_data = T9_signal_data[~np.isnan(T9_signal_data)]

    T10_signal_data = np.array(data_read['T10'])
    T10_signal_data = T10_signal_data[~np.isnan(T10_signal_data)]
    
    TP9_signal_data = np.array(data_read['TP9'])
    TP9_signal_data = TP9_signal_data[~np.isnan(TP9_signal_data)]

    TP10_signal_data = np.array(data_read['TP10'])
    TP10_signal_data = TP10_signal_data[~np.isnan(TP10_signal_data)]
    
    T_signal_data = T9_signal_data - T10_signal_data
    TP_signal_data = TP9_signal_data - TP10_signal_data

    sos = butter(16, np.array([0.5 , 120]) / (0.5*fs) , 'bandpass', output='sos')

    time_idx = 0
        
    T_signal_filtered = sosfiltfilt(sos, T_signal_data)
    # Spectrogram
    f, t, Sxx = signal.spectrogram(T_signal_filtered, fs, nperseg = n_perseg, noverlap = 0.5*n_perseg, 
                                       nfft=n_fft, scaling = 'density', mode='psd')
    Sxx_n = Sxx[0:out_shape,:]
    # Z-scoring normalization
    norm_Sxx = minmax_norm(stats.zscore(Sxx_n))
    T_Sxx = np.moveaxis(window_stack(norm_Sxx, stepsize=n_step, width = out_shape),2,0)

    TP_signal_filtered = sosfiltfilt(sos, TP_signal_data)
    # Spectrogram
    f, t, Sxx = signal.spectrogram(TP_signal_filtered, fs, nperseg = n_perseg, noverlap = 0.5*n_perseg, 
                                       nfft=n_fft, scaling = 'density', mode='psd')

    Sxx_n = Sxx[0:out_shape,:]
    # Z-scoring normalization
    norm_Sxx = minmax_norm(stats.zscore(Sxx_n))
    TP_Sxx = np.moveaxis(window_stack(norm_Sxx, stepsize=n_step, width = out_shape),2,0)

    X = np.stack((T_Sxx,TP_Sxx), axis = 3)
    X_out= np.concatenate([X, np.expand_dims(np.mean(X, axis = 3), axis = 3)], axis = 3)
#     print('X_out shape = ' + str(X_out.shape))
    
# get the end time
et = time.process_time()

# get execution time
res = et - st
final_res = res * 1000

print('CPU Execution time:', final_res, 'milliseconds')

#     for slice_idx in range(0, X_out.shape[0]):
#         np.save(output_path + fname_id +'-{:06d}_norm'.format(time_idx),X_out[slice_idx,:,:,:])
#         time_idx += 1
    

# print('*'*20 + 'DONE' + '*'*20)

CPU Execution time: 5226.898894 milliseconds
CPU Execution time: 5189.794678 milliseconds
CPU Execution time: 4983.260861 milliseconds
CPU Execution time: 4892.4862520000015 milliseconds
CPU Execution time: 5068.796920000001 milliseconds
CPU Execution time: 5489.945000999999 milliseconds
CPU Execution time: 6002.205603 milliseconds
CPU Execution time: 5034.895577 milliseconds
5.29 s ± 350 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
